In [1]:
from netCDF4 import Dataset
import numpy as np
import datetime
import matplotlib.pyplot as plt
from regrid import regrid
import mask
import tqdm
import cartopy.crs as ccrs
import cartopy
import matplotlib.animation as animation
from IPython import display
type_dir = '/home/robbie/Dropbox/SM_Thickness/data/CDS_type/'
        

/home/robbie/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/robbie/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [2]:
ice_type = Dataset('/home/robbie/Dropbox/SM_Thickness/data/CDS_type/2005/04/ice_type_nh_ease2-250_cdr-v1p0_200504041200.nc')
lon = ice_type['lon']
lat = ice_type['lat']

# Import data

In [11]:
d = Dataset('/home/robbie/Dropbox/daily_SIT/data/sden.nc')
sden = Dataset('/home/robbie/Dropbox/daily_SIT/data/sden.nc')['Snow Density']
snod = Dataset('/home/robbie/Dropbox/daily_SIT/data/snod.nc')['Snow Depth']
RFs = Dataset('/home/robbie/Dropbox/daily_SIT/data/CS2_CPOM_daily_RF.nc')
ice_type = np.load('/home/robbie/Dropbox/daily_SIT/data/icetype.npy').copy()
SWEs = np.multiply(snod,sden)

/tmp/ipykernel_17123/1217660331.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  SWEs = np.multiply(snod,sden)


In [12]:
snow_factor = 0.001813
rho_w   = 1023.9
rho_fyi = 916.7
rho_myi = 882

In [13]:
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=np.int(days)) for days in d['Day']]

/tmp/ipykernel_17123/232055474.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dates = [datetime.date(2010,1,1)+datetime.timedelta(days=np.int(days)) for days in d['Day']]
/tmp/ipykernel_17123/232055474.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1

In [30]:
years = np.arange(2010,2021)
yearly_data = {}
for year in years:
    yearly_data[year] = np.full((367,160,160),np.nan)
    
yearly_data = np.full((4018,160,160),np.nan)

for day, date in tqdm.tqdm(list(zip(range(d['Day'].shape[0]),dates))):
    
    RF = RFs['Radar Freeboard'][day]
    
    ice_type_r = ice_type[day]
    
    
    if ~np.array(np.isnan(RF)).all():
        
        if ~np.array(np.isnan(RF)).all():
            
    
#             ice_type_r[ice_type_r>3] = np.nan
#             ice_type_r[ice_type_r<2] = np.nan            
            ice_type_r[ice_type_r==2] = rho_fyi
            ice_type_r[ice_type_r==3] = rho_myi

            SWE = SWEs[day]

            density_field = ice_type_r

            hydro_coeff_field = np.divide(rho_w, np.subtract(rho_w,density_field))

            snow_coeff_field = np.multiply(snow_factor, hydro_coeff_field)

            SM_conts_field = np.multiply(SWE,snow_coeff_field)
            RF_conts_field = np.multiply(RF,hydro_coeff_field)
            SIT_field = SM_conts_field + RF_conts_field

            year = date.year
            doy = date.timetuple().tm_yday

#             yearly_data[year][doy] = SIT_field
            yearly_data[day] = SIT_field
            
#             plt.imshow(SIT_field)
#             plt.colorbar()
#             plt.show()
            
#             break

  0%|                                                                                                     | 0/4018 [00:00<?, ?it/s]/tmp/ipykernel_17123/1802703863.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  RF = RFs['Radar Freeboard'][day]
100%|████████████████████████████████████████████████████████████████████████████████████████| 4018/4018 [00:01<00:00, 2228.75it/s]


In [48]:
val_days = []
val_dates = []
for day in range(4018):
    sl = yearly_data[day]
    if ~np.isnan(sl).all():
        val_days.append(day)
        val_dates.append(dates[day])
#         plt.imshow(yearly_data[day])
#         plt.show()
val_days

[277,
 278,
 279,
 280,
 281,
 282,
 283,
 285,
 286,
 287,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447

In [49]:
val_dates

[datetime.date(2010, 10, 5),
 datetime.date(2010, 10, 6),
 datetime.date(2010, 10, 7),
 datetime.date(2010, 10, 8),
 datetime.date(2010, 10, 9),
 datetime.date(2010, 10, 10),
 datetime.date(2010, 10, 11),
 datetime.date(2010, 10, 13),
 datetime.date(2010, 10, 14),
 datetime.date(2010, 10, 15),
 datetime.date(2010, 10, 17),
 datetime.date(2010, 10, 18),
 datetime.date(2010, 10, 19),
 datetime.date(2010, 10, 20),
 datetime.date(2010, 10, 21),
 datetime.date(2010, 10, 22),
 datetime.date(2010, 10, 23),
 datetime.date(2010, 10, 24),
 datetime.date(2010, 10, 25),
 datetime.date(2010, 10, 26),
 datetime.date(2010, 10, 27),
 datetime.date(2010, 10, 28),
 datetime.date(2010, 10, 30),
 datetime.date(2010, 10, 31),
 datetime.date(2010, 11, 1),
 datetime.date(2010, 11, 2),
 datetime.date(2010, 11, 3),
 datetime.date(2010, 11, 4),
 datetime.date(2010, 11, 5),
 datetime.date(2010, 11, 6),
 datetime.date(2010, 11, 7),
 datetime.date(2010, 11, 8),
 datetime.date(2010, 11, 9),
 datetime.date(2010, 11,

In [52]:
d['Longitude']

<class 'netCDF4._netCDF4.Variable'>
float64 Longitude(x, y)
    _FillValue: nan
unlimited dimensions: 
current shape = (160, 160)
filling on

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(8,8),
                    subplot_kw={'projection':ccrs.NorthPolarStereo()})    

ax.add_feature(cartopy.feature.LAND, edgecolor='black',zorder=0,alpha=0.8)

ax.set_extent([-180, 180, 90, 66], ccrs.PlateCarree())

bg = ax.pcolormesh(np.array(d['Longitude']),
                    np.array(d['Latitude']),
                   yearly_data[0][:-1,:-1],
                   vmin = 0, vmax = 3,
                     transform=ccrs.PlateCarree(),
                   cmap='plasma',
                   zorder=5,
                   alpha=1)


anno = ax.annotate('',xycoords='axes fraction',xy=(0.99,0.93),ha='right',fontsize='x-large',zorder=7)


def animate(frame):
    
    if frame %50 == 0: print(frame)

    bg.set_array(yearly_data[frame][:-1,:-1])
    anno.set_text(val_dates[frame])

#######################################################

fig.subplots_adjust(left=0.01, bottom=0, right=0.99, top=1, wspace=None, hspace=None)


ani = animation.FuncAnimation(fig,
                              animate,
                             frames= val_days[:],
                             )   

# video = ani.to_html5_video()
# html = display.HTML(video)
# display.display(html)
# plt.close()
f = 'good_frames.mp4'
writervideo = animation.FFMpegWriter(fps=10) 
ani.save(f, writer=writervideo)

/tmp/ipykernel_17123/3569027364.py:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bg = ax.pcolormesh(np.array(d['Longitude']),
/tmp/ipykernel_17123/3569027364.py:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.array(d['Latitude']),


300
350
400
450
650


In [40]:
d

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): t(4018), x(160), y(160)
    variables(dimensions): float64 Snow Density(t,x,y), float64 Longitude(x,y), float64 Latitude(x,y), int64 Day(t)
    groups: 